# Automatic Differentiation and Taylor Series Expansions for Solving ODEs

In [1]:
using Test

## Solving ODEs Numerically

Ordinary Differential Equations (ODEs) are known to be difficult (and at times impossible) to solve analytically. However, there are numerous methods to numerically approximate the solution of an ODE, given an initial value. In this section, we are concerned with the algorithmic approximation of the solution of a given initial value problem (IVP):

$$
\frac{dx}{dt} = f\left(t, x \right), \  x\left(t_0\right) = x_0
$$

One method used for approximating such IVPs is to make use of their Taylor Series expansions. We first require the RHS of the differential equation ($f$) to be smooth, and thus its Taylor Series expansion exists.Any solution to this differential equation $\mu$ satisfies $\frac{d\mu}{dt} = f\left(t, \mu \right)$ and $\mu\left(t_0\right) = \mu_0 (=x_0)$, and has Taylor Series expansion centered at $t_0$:

$$
\mu\left(t\right) = \mu\left(t_0\right) + \mu'\left(t_0\right)\left(t - t_0\right) + \frac{\mu''\left(t_0\right)}{2!}\left(t - t_0\right)^2 + \cdots + \frac{\mu^{\left(k\right)}\left(t_0\right)}{k!}\left(t - t_0\right)^k
$$

Where we have $\mu'\left(t_0\right) = \mu^{\left(1\right)}\left(t_0\right) = \frac{d\mu}{dt}\left(t_0\right)$. Our goal is to derive the coefficients for each term in the Taylor Series expansion (up to a desired order in practice) to approximate the solution about the initial value. We note that the Taylor Series expansion of $f$ centered at $t_0$ can be derived in a similar fashion. By taking the derivative of the Taylor Series expansion of $\mu$, we can equate the two Taylor Series expansions via the differential equation. Comparing term by term (coeffficients of $\left(t-t_0\right)^k$), we have the following equations:

$$
\mu^{\left(k\right)}\left(t_0\right) = \left(f\circ\mu\right)^{\left(k-1\right)}(t_0) \ \ \forall k > 0
$$
$$
\begin{cases}
\mu\left(t_0\right) &= \mu_0 \\
\mu'\left(t_0\right) &= f\left(t_0, x_0\right) \\
\mu''\left(t_0\right) &= \frac{d}{dt}f\left(t_0, x_0\right) = \frac{\partial f}{\partial t} + \frac{\partial f}{\partial x}f\left(t_0, x_0\right) \\
\mu'''\left(t_0\right) &= \cdots = \frac{\partial^2 f}{\partial t^2} + 2\frac{\partial^2 f}{\partial t\partial x}f\left(t_0, x_0\right) + \frac{\partial^2 f}{\partial x^2}\left(f\left(t_0, x_0\right)\right)^2 + \frac{\partial f}{\partial x}\left(\frac{\partial f}{\partial t} + \frac{\partial f}{\partial x}f\left(t_0, x_0\right)\right)\\
&\vdots
\end{cases}
$$

Where we have shortened $\frac{\partial f}{\partial t} := \frac{\partial f}{\partial t}\left(t_0, x_0\right)$ as the evaluation points are implied. There are numerous methods to go about approximating each value, but as the expressions for higher derivatives of $\left(f\circ\mu\right)$ becomes increasingly tedious, we require efficient methods of calculation while maintaining a high degree of accuracy. We note that making use of divided difference tends to incur large errors, and recalculating symbolic expressions for higher derivatives is memory and time consuming (as demonstrated above). One method that attempts to minimise errors and memory usage is via automatic differentiation - specifically, the use of dual numbers in high dimensional differentiation.

## Dual Numbers 

The group of simple one-dimensional dual number (EDITOR NOTE: find a proper name for this, in comparison to what is presented below) is familiar: $\mathbb{D} := \{ a + b\epsilon | a, b \in \mathbb{R}, \epsilon^2 = 0 \}$. Such dual numbers can be used to calculate the first derivative of a given function evaluated at a point: $f\left(a + b\epsilon\right) = f\left(a\right) + bf'\left(a\right)\epsilon$. This same idea can be extended in two ways: to higher order derivatives, as well as in multi-variable functions, which will prove useful in numerically solving ODEs.

### Extending Dual Numbers: Higher Order Derivatives
...

### Extending Dual Numbers: Multi-variable Functions
...

In [4]:
N = 2
struct Dual
    a
    et
    ex
    et2
    etx
    ex2
end

Dual(x::Number) = Dual(x, 0, 0, 0, 0, 0)
@test Dual(3) == Dual(3, 0, 0, 0, 0, 0)

Test Passed

In [35]:
import Base: +, -, *, /

function +(x::Dual, y::Dual)
    Dual(x.a + y.a, x.et + y.et, x.ex + y.ex, x.et2 + y.et2, x.etx + y.etx,
         x.ex2 + y.ex2)
end

-(x::Dual) = Dual(-x.a, -x.et, -x.ex, -x.et2, -x.etx, -x.ex2)

function -(x::Dual, y::Dual)
    x + -y
end

function *(x::Dual, y::Dual)
    Dual(x.a*y.a, x.a*y.et+y.a*x.et, x.a*y.ex+y.a*x.ex,
         2*x.et*y.et+x.a*y.et2+y.a*x.et2,
         x.ex*y.et+x.et*y.ex+x.a*y.etx+y.a*x.etx,
         2*x.ex*y.ex+x.a*y.ex2+y.a*x.ex2)
end

*(x::Number, y::Dual) = Dual(x)*y

#function /(x::Dual, y::Dual)   

@test Dual(1, 2, 3, 4, 5, 6) + Dual(4, 4, 4, 4, 4, 4) == Dual(5, 6, 7, 8, 9, 10)
@test -Dual(1, 2, 3, 4, 5, 6) == Dual(-1, -2, -3, -4, -5, -6)
@test Dual(1, 2, 3, 4, 5, 6) - Dual(4, 4, 4, 4, 4, 4) == Dual(-3, -2, -1, 0, 1, 2)
@test Dual(1, 0, 1, 0, 0, 0) * Dual(1, 1, 0, 0, 0, 0) == Dual(1, 1, 1, 0, 1, 0)

Test Passed

In [36]:
import Base: exp, ^

function exp(x::Dual)
    expx = exp(x.a)
    Dual(expx, x.et*expx, x.ex*expx, (x.et2 + (x.et)^2)*expx,
         (x.etx + x.et*x.ex)*expx, (x.ex2 + (x.ex^2))*expx)
end

@test exp(Dual(1, 1, 1, 0, 1, 0)) == Dual(exp(1), exp(1), exp(1), exp(1), 2*exp(1), exp(1))

Test Passed

In [37]:
function f(t, x)
    exp(2*t-x)
end
vals = f(Dual(1, 1, 0, 0, 0, 0), Dual(1, 0, 1, 0, 0, 0))

Dual(2.718281828459045, 5.43656365691809, -2.718281828459045, 10.87312731383618, -5.43656365691809, 2.718281828459045)

In [38]:
a, et, exe, et2, etx, ex2 = vals.a, vals.et, vals.ex, vals.et2, vals.etx, vals.ex2

mu0 = 1
mu1 = a
mu2 = et + exe
mu3 = et2 + 2*etx*a + ex2*a^2 + exe*(et + exe*a)

(mu0, mu1, mu2, mu3)


(1, 2.718281828459045, 2.718281828459045, 6.709864566627615)